# Realistic Chunking Strategy Comparison

This notebook compares chunking strategies using realistic Claude Code documentation content with German/English cross-language queries.

In [64]:
import sys
import os
sys.path.insert(0, '/Users/florianwegener/Projects/crawl4ai-mcp-server')

from tools.knowledge_base.dependencies import is_rag_available
print("RAG available:", is_rag_available())

RAG available: True


In [65]:
# Initialize the Enhanced Content Processor
from tools.knowledge_base.enhanced_content_processor import EnhancedContentProcessor

processor = EnhancedContentProcessor(
    chunking_strategy="auto",
    chunk_size=1000,
    chunk_overlap=100,
    enable_ab_testing=True,
    quality_threshold=0.7
)

print("✅ Enhanced Content Processor initialized")
print(f"Strategy: {processor.chunking_strategy}")
print(f"Chunk size: {processor.chunk_size}")
print(f"Overlap: {processor.chunk_overlap}")

✅ Enhanced Content Processor initialized
Strategy: auto
Chunk size: 1000
Overlap: 100


In [66]:
# Sample Claude Code documentation content
claude_code_content = '''
# Claude Code Memory Management

Claude Code provides several types of memory management to help you work more efficiently:

## Conversation Memory

Your conversation with Claude persists throughout a session, allowing Claude to reference earlier parts of your conversation. This includes:

- Code you've written together
- Files you've explored
- Problems you've solved
- Context about your project

### Memory Limitations

```bash
# Check current memory usage
claude --memory-status

# Clear conversation memory
claude --clear-memory
```

## Project Context (CLAUDE.md)

The `CLAUDE.md` file serves as persistent project memory. It should contain:

1. **Project Overview**: What your project does
2. **Architecture**: Key components and their relationships  
3. **Development Workflow**: Common commands and procedures
4. **Important Files**: Critical files Claude should know about

### CLAUDE.md Best Practices

| Section | Purpose | Example |
|---------|---------|----------|
| Overview | High-level description | "React app with Node.js backend" |
| Commands | Essential commands | `npm run dev`, `npm test` |
| Architecture | Key directories/files | `/src/components`, `/api/routes` |

```markdown
# My Project

## Overview
This is a full-stack web application...

## Essential Commands
- `npm run dev`: Start development server
- `npm test`: Run test suite
```

## File Context

Claude automatically maintains context about files you're working with:

- **Recently accessed files**: Files you've read or modified
- **Import relationships**: Dependencies between files
- **Code structure**: Functions, classes, and exports

### Context Optimization

To optimize Claude's understanding:

```python
def important_function():
    """This function is critical to the application.
    
    Make sure Claude knows about its purpose and usage.
    """
    pass
```

## Memory Types Summary

1. **Conversation Memory**: Session-based, temporary
2. **Project Memory (CLAUDE.md)**: Persistent, version-controlled
3. **File Context**: Automatic, based on file access patterns

Each type serves different purposes in helping Claude understand and work with your codebase effectively.
'''

print(f"Sample content length: {len(claude_code_content)} characters")
print("\n--- Content Preview ---")
print(claude_code_content[:200] + "...")

Sample content length: 2176 characters

--- Content Preview ---

# Claude Code Memory Management

Claude Code provides several types of memory management to help you work more efficiently:

## Conversation Memory

Your conversation with Claude persists throughout ...


In [67]:
# Process content with both strategies for comparison
result = processor.compare_strategies(
    content=claude_code_content,
    source_metadata={'title': 'Claude Code Memory Management'}
)

print("=== Chunking Strategy Comparison Results ===")
print(f"Baseline chunks: {result.baseline_chunks}")
print(f"Enhanced chunks: {result.enhanced_chunks}")
print(f"Baseline time: {result.baseline_time:.3f}s")
print(f"Enhanced time: {result.enhanced_time:.3f}s")
print(f"Baseline avg size: {result.baseline_avg_size:.1f} chars")
print(f"Enhanced avg size: {result.enhanced_avg_size:.1f} chars")
print(f"Semantic preservation: {result.semantic_preservation_score:.3f}")
print(f"Quality improvement: {result.quality_improvement_ratio:.3f}")
print(f"Performance ratio: {result.performance_ratio:.3f}")
print(f"Recommendation: {result.recommendation}")

=== Chunking Strategy Comparison Results ===
Baseline chunks: 3
Enhanced chunks: 8
Baseline time: 0.000s
Enhanced time: 0.001s
Baseline avg size: 758.0 chars
Enhanced avg size: 269.5 chars
Semantic preservation: 0.770
Quality improvement: 0.770
Performance ratio: 0.394
Recommendation: markdown_intelligent


In [74]:
# Process with the recommended strategy to get chunks for embedding
chunks = processor.process_content(
    content=claude_code_content,
    source_metadata={'title': 'Claude Code Memory Management'},
    force_strategy=result.recommendation
)

print(f"\n=== Processing with {result.recommendation} strategy ===")
print(f"Generated {len(chunks)} chunks")

# Show chunk details
for i, chunk in enumerate(chunks[:3]):  # Show first 3 chunks
    print(f"\n--- Chunk {i+1} ---")
    print(f"Content length: {len(chunk['content'])} chars")
    print(f"Strategy: {chunk['metadata'].get('chunking_strategy')}")
    print(f"Header hierarchy: {chunk['metadata'].get('header_hierarchy')}")
    print(f"Contains code: {chunk['metadata'].get('contains_code')}")
    print(f"Preview: {chunk['content'][:150]}...")


=== Processing with markdown_intelligent strategy ===
Generated 8 chunks

--- Chunk 1 ---
Content length: 124 chars
Strategy: markdown_intelligent
Header hierarchy: ['Claude Code Memory Management']
Contains code: False
Preview: # Claude Code Memory Management  
Claude Code provides several types of memory management to help you work more efficiently:...

--- Chunk 2 ---
Content length: 276 chars
Strategy: markdown_intelligent
Header hierarchy: ['Claude Code Memory Management', 'Conversation Memory']
Contains code: False
Preview: ## Conversation Memory  
Your conversation with Claude persists throughout a session, allowing Claude to reference earlier parts of your conversation....

--- Chunk 3 ---
Content length: 139 chars
Strategy: markdown_intelligent
Header hierarchy: ['Claude Code Memory Management', 'Conversation Memory', 'Memory Limitations']
Contains code: True
Preview: ### Memory Limitations  
```bash
# Check current memory usage
claude --memory-status

# Clear conversation me

In [69]:
# Show ALL chunks with FULL content
print(f"\n=== COMPLETE CHUNK BREAKDOWN ===")
print(f"Total chunks: {len(chunks)}")

for i, chunk in enumerate(chunks):
    print(f"\n{'='*60}")
    print(f"CHUNK {i+1} of {len(chunks)}")
    print(f"{'='*60}")
    print(f"Length: {len(chunk['content'])} characters")
    print(f"Strategy: {chunk['metadata'].get('chunking_strategy')}")
    print(f"Header hierarchy: {chunk['metadata'].get('header_hierarchy')}")
    print(f"Contains code: {chunk['metadata'].get('contains_code')}")
    print(f"Programming language: {chunk['metadata'].get('programming_language')}")
    print(f"\nFULL CONTENT:")
    print("-" * 40)
    print(repr(chunk['content']))  # Use repr to show exact content including whitespace
    print("-" * 40)


=== COMPLETE CHUNK BREAKDOWN ===
Total chunks: 8

CHUNK 1 of 8
Length: 124 characters
Strategy: markdown_intelligent
Header hierarchy: ['Claude Code Memory Management']
Contains code: False
Programming language: None

FULL CONTENT:
----------------------------------------
'# Claude Code Memory Management  \nClaude Code provides several types of memory management to help you work more efficiently:'
----------------------------------------

CHUNK 2 of 8
Length: 276 characters
Strategy: markdown_intelligent
Header hierarchy: ['Claude Code Memory Management', 'Conversation Memory']
Contains code: False
Programming language: None

FULL CONTENT:
----------------------------------------
"## Conversation Memory  \nYour conversation with Claude persists throughout a session, allowing Claude to reference earlier parts of your conversation. This includes:  \n- Code you've written together\n- Files you've explored\n- Problems you've solved\n- Context about your project"
--------------------------

In [70]:
# Test cross-language semantic similarity
from tools.knowledge_base.embeddings import EmbeddingService

embedding_service = EmbeddingService()
print(f"Using embedding model: {embedding_service.model_name}")

# Create embeddings for all chunks
chunk_embeddings = []
chunk_texts = []

for chunk in chunks:
    embedding = embedding_service.encode_text(chunk['content'])
    chunk_embeddings.append(embedding)
    chunk_texts.append(chunk['content'])

print(f"\nCreated embeddings for {len(chunk_embeddings)} chunks")
print(f"Embedding dimension: {len(chunk_embeddings[0])}")

Using embedding model: distiluse-base-multilingual-cased-v1

Created embeddings for 8 chunks
Embedding dimension: 512


In [71]:
# Test cross-language semantic similarity
from tools.knowledge_base.embeddings import EmbeddingService

embedding_service = EmbeddingService()
print(f"Using embedding model: {embedding_service.model_name}")

# Create embeddings for all chunks
chunk_embeddings = []
chunk_texts = []

for chunk in chunks:
    embedding = embedding_service.encode_text(chunk['content'])
    chunk_embeddings.append(embedding)
    chunk_texts.append(chunk['content'])

print(f"\nCreated embeddings for {len(chunk_embeddings)} chunks")
print(f"Embedding dimension: {len(chunk_embeddings[0])}")

Using embedding model: distiluse-base-multilingual-cased-v1

Created embeddings for 8 chunks
Embedding dimension: 512


In [72]:
# Test German/English cross-language queries
import numpy as np
from scipy.spatial.distance import cosine

# Define test query pairs (German/English)
query_pairs = [
    ("Welche verschiedenen Arten von Speicher gibt es in Claude Code?", 
     "What are the different types of memory in Claude Code?"),
    
    ("Wie funktioniert die CLAUDE.md Datei?",
     "How does the CLAUDE.md file work?"),
    
    ("Was sind die besten Praktiken für Projektkontext?",
     "What are the best practices for project context?"),
    
    ("Welche Befehle kann ich verwenden um den Speicher zu verwalten?",
     "What commands can I use to manage memory?")
]

print("=== Cross-Language Semantic Similarity Test ===")

for i, (german_query, english_query) in enumerate(query_pairs, 1):
    print(f"\n--- Query Pair {i} ---")
    print(f"German: {german_query}")
    print(f"English: {english_query}")
    
    # Get embeddings for both queries
    german_emb = embedding_service.encode_text(german_query)
    english_emb = embedding_service.encode_text(english_query)
    
    # Calculate cross-language similarity
    cross_lang_similarity = 1 - cosine(german_emb, english_emb)
    print(f"Cross-language similarity: {cross_lang_similarity:.3f}")
    
    # Find best matching chunks for each query
    german_similarities = [1 - cosine(german_emb, chunk_emb) for chunk_emb in chunk_embeddings]
    english_similarities = [1 - cosine(english_emb, chunk_emb) for chunk_emb in chunk_embeddings]
    
    best_german_idx = np.argmax(german_similarities)
    best_english_idx = np.argmax(english_similarities)
    
    print(f"Best match for German: Chunk {best_german_idx} (similarity: {german_similarities[best_german_idx]:.3f})")
    print(f"Best match for English: Chunk {best_english_idx} (similarity: {english_similarities[best_english_idx]:.3f})")
    
    # Check if both languages find the same most relevant chunk
    same_chunk = best_german_idx == best_english_idx
    print(f"Same best chunk: {same_chunk} {'✅' if same_chunk else '❌'}")

=== Cross-Language Semantic Similarity Test ===

--- Query Pair 1 ---
German: Welche verschiedenen Arten von Speicher gibt es in Claude Code?
English: What are the different types of memory in Claude Code?
Cross-language similarity: 0.881
Best match for German: Chunk 0 (similarity: 0.595)
Best match for English: Chunk 0 (similarity: 0.581)
Same best chunk: True ✅

--- Query Pair 2 ---
German: Wie funktioniert die CLAUDE.md Datei?
English: How does the CLAUDE.md file work?
Cross-language similarity: 0.970
Best match for German: Chunk 3 (similarity: 0.454)
Best match for English: Chunk 3 (similarity: 0.469)
Same best chunk: True ✅

--- Query Pair 3 ---
German: Was sind die besten Praktiken für Projektkontext?
English: What are the best practices for project context?
Cross-language similarity: 0.910
Best match for German: Chunk 3 (similarity: 0.378)
Best match for English: Chunk 3 (similarity: 0.406)
Same best chunk: True ✅

--- Query Pair 4 ---
German: Welche Befehle kann ich verwenden u

In [73]:
# Summary and analysis
print("\n" + "=" * 60)
print("SUMMARY: Realistic Chunking Comparison with Cross-Language Testing")
print("=" * 60)

print(f"\n📊 Chunking Performance:")
print(f"   Strategy recommendation: {result.recommendation}")
print(f"   Quality improvement: {result.quality_improvement_ratio:.1%}")
print(f"   Performance ratio: {result.performance_ratio:.2f}x")

print(f"\n🌍 Cross-Language Analysis:")
all_cross_similarities = []

for german_query, english_query in query_pairs:
    german_emb = embedding_service.encode_text(german_query)
    english_emb = embedding_service.encode_text(english_query)
    similarity = 1 - cosine(german_emb, english_emb)
    all_cross_similarities.append(similarity)

avg_cross_similarity = np.mean(all_cross_similarities)
print(f"   Average cross-language similarity: {avg_cross_similarity:.3f}")
print(f"   Model: {embedding_service.model_name}")
print(f"   Embedding dimensions: {len(chunk_embeddings[0])}D")

print(f"\n🎯 Content Processing:")
print(f"   Total chunks generated: {len(chunks)}")
print(f"   Average chunk size: {np.mean([len(c['content']) for c in chunks]):.0f} chars")
print(f"   Strategy used: {chunks[0]['metadata']['chunking_strategy']}")

# Performance assessment
if avg_cross_similarity > 0.8:
    assessment = "🟢 Excellent cross-language performance"
elif avg_cross_similarity > 0.7:
    assessment = "🟡 Good cross-language performance"
else:
    assessment = "🔴 Poor cross-language performance"

print(f"\n{assessment}")
print(f"✅ Notebook completed successfully!")


SUMMARY: Realistic Chunking Comparison with Cross-Language Testing

📊 Chunking Performance:
   Strategy recommendation: markdown_intelligent
   Quality improvement: 77.0%
   Performance ratio: 0.39x

🌍 Cross-Language Analysis:
   Average cross-language similarity: 0.919
   Model: distiluse-base-multilingual-cased-v1
   Embedding dimensions: 512D

🎯 Content Processing:
   Total chunks generated: 8
   Average chunk size: 270 chars
   Strategy used: markdown_intelligent

🟢 Excellent cross-language performance
✅ Notebook completed successfully!
